# Binary Full Adder in Keras

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/luckykadam/adder/blob/master/full_adder.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/luckykadam/adder/blob/master/full_adder.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

## Background

Neurons (with non-linear activation) are only capable of learning a linear decision boundary, but when sufficient units and layers of neurons work together, they can learn theoretically any complex function. This property of NNs has been mathematically proven, and demonstrated via many examples.

To understand the fundamentals of how can neural networks achieve this, it is more beneficial for a new-comer to analyse the individual parameters, and how they contribute to the final output, instead of starting with complex task like cat/dog image classification, which might be overwhelming.

Today, we will build a very simple neural network, to simlulate the binary full adder and analyse its parameters.

## Binary Adder

Binary Adder is a circuit to add two binary numbers.

### Half Adder

The Half Adder is used to add two binary bits. The half adder outputs the sum of two input bits and a carry value.

<img height="100" src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d9/Half_Adder.svg/360px-Half_Adder.svg.png">

### Full Adder

A Full Adder can perform an addition operation on three bits. The full adder produces a sum of three inputs and carry value. The carry value can then be used as input to the next full adder.

Using this unit in repeatition, two binary numbers of arbitrary length can be added.

<img height="180" src="https://upload.wikimedia.org/wikipedia/commons/thumb/6/69/Full-adder_logic_diagram.svg/800px-Full-adder_logic_diagram.svg.png">


## Logic Gates

Some logic gates which have a linear decision boundary(eg. AND, OR, NAND, NOR) can be represented using a single neuron, whereas some logic gates don't have a linear decision boundary(eg. XOR) and require multiple neurons or layers.

TODO: clarify the number of neurons required for different types of logic gates
<p>
TODO: insert images to show decision boundary of different gates

## Implementation

We are going to use Keras (`tf.keras`) to create a neural network that can simulate full adder.

In [1]:
import numpy as np

import tensorflow as tf
from tensorflow.keras import models, layers, activations

np.random.seed(0)
tf.random.set_seed(0)

## Dataset creation

There are only 8 possible combination to inputs and ouputs. To create the dataset, we are going to repeat these unique samples.

In [2]:
n_samples = 100000//8

In [3]:
x = np.array([[0, 0, 0], [0, 1, 0], [1, 0, 0], [1, 1, 0], [0, 0, 1], [0, 1, 1], [1, 0, 1], [1, 1, 1]] * n_samples) # a, b, c
y = np.array([[0, 0], [0, 1], [0, 1], [1, 0], [0, 1], [1, 0], [1, 0], [1, 1]] * n_samples) # c, s

sk-learn provides great functionality to split the dataset into train and test samples.

In [4]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, shuffle=True)

## Model

We model the problem as classification of sum bit as active/inactive and carry bit as active/inactive.

Keras provides multiple ways to define a model.

1. `Sequential` API provides an easy interface to create a model, where data flows sequentially through a stack of layers.
2. `Model` Functional API can define complex models, such as multi-output models, directed acyclic graphs, or models with shared layers.
3. To get even more control, Keras allows to extend `Model` class and override `__init__` and `call` functions.

In our example we don't want any feedback loop, hence `Sequencial` model would work. The layers in out network are:

1. `Input`: Inputs to the network are 3 bits: a, b and c (carry).
2. `Dense`: TODO: justify dimension of hidden layer
3. `Dense`: Outputs are 2 bits: c (next carry), s (sum).

In [5]:
model = models.Sequential(name='full_adder')
model.add(layers.Input(shape=(3,), name='input'))
model.add(layers.Dense(units=3, activation=activations.tanh, name='hidden'))
model.add(layers.Dense(units=2, activation=activations.sigmoid, name='output'))

model.summary()

Model: "full_adder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden (Dense)               (None, 3)                 12        
_________________________________________________________________
output (Dense)               (None, 2)                 8         
Total params: 20
Trainable params: 20
Non-trainable params: 0
_________________________________________________________________


## Loss function

Choice of loss function is a crucial step. The most common choices for different types of problems are:

1. **Regression**: Default choice is `mean_squared_error`.
2. **Classification**:
  * **Binary Classification**: Default choice is `binary_crossentropy`. The function requires that the output layer is configured with a `sigmoid` activation to limit output to (0, 1).
  * **Multi-Class Single-Label Classification**: When a sample can belong to any one class (of >2 options), default choice is `categorical_crossentropy`. This also requires a `sigmoid` activated output layer.
  * **Multi-Class Multi-Label Classification**: When a sample might belong to multiple classes (of >1 options), default choice is `binary_crossentropy`. Here each classification is considered independent of each other, and effectively multiple binary classifiers are learned at once. This also requires a `sigmoid` activated output layer.

In out Full Adder model both the outputs can be '1' at the same time, hence it would be a multi-class multi-label classification.

On choosing optimizer, `adam` works well in most cases.

In [6]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

## Training

When training set is large, each iteration (parameter update step) might take very long. This way progress in parameters is very slow.

One trick to fasten this process is to consider small subset of train set, that fairly represents the complete train set, for each iteration. That way iterations would complete in short durations, and parameters is would be updated more frequently.

How would we create those small subset? Simplest way is to shuffle the train set and select subsets sequentially.

This trick is called **Mini-Batching**. In keras default method is mini batching controlled by `mini_batch` parameter of `Model.fit()` method. To disable mini-batching, set `batch_size` parameter to size of train set.


In [7]:
model.fit(x_train, y_train, batch_size=32, epochs=3)
scores = model.evaluate(x_test, y_test, verbose=2)
print(scores)

Train on 90000 samples
Epoch 1/3
90000/90000 [==============================] - 4s 44us/sample - loss: 0.4455 - accuracy: 0.8045
Epoch 2/3
90000/90000 [==============================] - 4s 41us/sample - loss: 0.1598 - accuracy: 0.9767
Epoch 3/3
90000/90000 [==============================] - 3s 32us/sample - loss: 0.0315 - accuracy: 1.0000
10000/1 - 0s - loss: 0.0154 - accuracy: 1.0000
[0.01452168820798397, 1.0]


## Testing

Let's write some code to add two numbers of arbitraty length.

Generate two random numbers in range (0, 2<sup>`max_bits`-1</sup>), so that total sum is in range (0, 2<sup>`max_bits`</sup>)

In [8]:
max_bits = 8

a = np.random.randint(np.power(2, max_bits-1))
b = np.random.randint(np.power(2, max_bits-1))

a_bin = 1 * (np.flip(list(np.binary_repr(a, width=max_bits)), axis=-1) == '1')
b_bin = 1 * (np.flip(list(np.binary_repr(b, width=max_bits)), axis=-1) == '1')

print('a: {}, b: {}'.format(a, b))
print('binary representations -> a: {}, b: {}'.format(a_bin, b_bin))

a: 18, b: 39
binary representations -> a: [0 1 0 0 1 0 0 0], b: [1 1 1 0 0 1 0 0]


Now, iterate over bits of both numbers, feeding them with carry bit to the model. The ouput at each step is used to evaluate the corresponding bit of summed value, and update the carry bit.

In [9]:
c = 0
predictions = np.zeros(max_bits)

for i in range(max_bits):
    a_b_c = np.array([[a_bin[i], b_bin[i], c]], dtype='float32')
    c_s = model(a_b_c)[0]
    c = c_s[0]
    predictions[i] = c_s[1]

print('predictions: {}'.format(predictions))

summed_bin = 1 * (predictions > 0.5)
print('binary representations -> summed: {}'.format(summed_bin))

summed = np.packbits(np.flip(summed_bin , axis=-1))[0]
print('summed: {}'.format(summed))

predictions: [0.98617655 0.03193694 0.01723741 0.96571255 0.97988051 0.98609447
 0.03419748 0.03421632]
binary representations -> summed: [1 0 0 1 1 1 0 0]
summed: 57


## Result

As we can see, our model works perfectly.

## Further Reading

Similar implementation of half adder can be found at: [github.com/luckykadam/adder/blob/master/half_adder.ipynb]

## References:

1. https://en.wikibooks.org/wiki/Digital_Electronics/Digital_Adder
2. https://towardsdatascience.com/neural-representation-of-logic-gates-df044ec922bc
3. https://machinelearningmastery.com/how-to-choose-loss-functions-when-training-deep-learning-neural-networks/